In [1]:
import random
import nltk
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import LinearSVC

In [2]:
BOT_CONFIG = {'intents': {
	'hello': {
		'examples': [
			'Hello', 'Hi', 'Hey', 'Good morning', 'Good afternoon', 'Hows it going?', 'Good evening', 'Hello again!', 'It’s good to see you again!', 'Howdy', 'Hiya', 'What’s up?', 'Yo!', 'Nice to see you', 'I haven’t seen you in a while', 'How are things?', 'What’s new?', 'How are you getting on?', 'Thanks for agreeing to meet with me', 'Have you been here long?', ' I don’t think we’ve met.', ' I’ve heard so much about you.', 'It’s nice to put a face to a name.'
		],
		'responses': [
			'Very well', 'Not bad', 'OK', 'Great', 'I am happy.Thx', 'I am feeling grateful.Thx', 'I have been better.Thx', 'Do you want the short or long story?', 'Gearing up for the week', 'Trying to stay alive.Thx', 'Better now that you are here.Thx', 'In need of some peace', 'Want to get away....Thx', 'I am wondering, how you are?', 'Same old', 'Oh, gosh, all kinds of stuff', 'Nothing new', 'Not a lot', 'Nothing too much', 'Oh, just the usual', 'Fine', 'Pretty good', 'Not terrific, not terrible, just average', 'I am hanging in there', 'Like a pitbull in a butcher shop'
		]},
	'bye': {
		'examples': [
			'Goodbye', 'Have a good day', 'Have a nice day', 'Have a good evening', 'Have a good night', 'Have a good one', 'Take care', 'Bye-bye', 'See you' 'Talk to you later', 'Catch you later', 'Farewell', 'Later', 'So long', 'All right then', 'Take it easy', 'Inabit', 'Peace!', 'Im out!', 'I gotta roll', 'Toodle-oo', 'Toodle-pip', 'See you later alligator', 'Godspeed', 'Adios', 'Ciao!', 'Au revoir', 'Sayonara!', 'Auf Wiedersehen'
		],
		'responses': [
			'Auf Wiedersehen', 'Sayonara!', 'Au revoir', 'Ciao!', 'Adios', 'Godspeed', 'See you later alligator', 'Toodle-pip', 'Toodle-oo', 'I gotta roll', 'Im out!', 'Peace!', 'Inabit', 'Мне пора', 'Take it easy', 'All right then', 'So long', 'Later', 'Farewell', 'Catch you later', 'See youTalk to you later', 'Bye-bye', 'Take care', 'Have a good one', 'Have a good night', 'Have a good evening', 'Have a nice day', 'Have a good day', 'Goodbye'
		]},
	'thanks': {
		'examples': [
			'THANKS', 'thx', 'THANK YOU', 'THANK YOU VERY MUCH', 'THANK YOU SO MUCH', 'THANK YOU. THAT’S VERY KIND OF YOU', 'I CAN’T THANK YOU ENOUGH', 'THANKS AWFULLY', 'TNANK YOU IN ADVANCE', 'MANY THANKS', 'Thanks a bunch', 'I really appreciate it', 'Much obliged', 'I owe you one', 'I can’t thank you enough', ' It’s so kind of you', 'This means a lot to me', 'I’m grateful for that'
		],
		'responses': [
			'You are very welcome', 'Don’t mention it', 'My pleasure', 'Not a problem', 'It was nothing!', 'Not at all', 'Sure', 'Anytime!', 'Don’t worry about it!', 'Anything for you!', 'I would have done the same for you', 'Happy to help!', 'It was the least I could do'
		]},
	'whatcanyoudo': {
		'examples': [
			'Tell me what you can do?', 'What can you do?', 'Tell me what you can', 'Tell me what you can do', 'What are you capable of?', 'Tell me what you can', 'Why are you need?', 'What can you do?', 'Tell me what you can do', 'Hfccrf; b xnj evttim', 'What can you do?', 'Can you do a lot?', 'What are your functions?'
		],
		'responses': [
			'I talk a lot', 'Answer questions. So far, nothing else. ',' I definitely dont know how to tie shoelaces', 'I understand the sorts of beer', 'Answer questions',' I can tell about myself ',' I can advise a movie ',' I am a simple interlocutor'
		]},
	'name': {
		'examples': [
			'Say your name', 'What is your name?', 'Do you have a name?', 'How to dignify you?', 'Tell me what is your name?', 'Introduce yourself', 'What is your name?', 'Your name', 'What are you?', 'What to call you', 'Tell me your name', 'What is your name?', 'Your name?', 'Lets get to know you', 'Name?', 'Who are you?'
		],
		'responses': [
			'If you want, call me a bot', 'My name is a bot. Just a bot ',' I dont have a name, Im just a program '' The name doesnt matter ',' Lets not talk about the name'
		]},
	'weather': {
		'examples': [
			'What is the weather', 'Tell me the weather?', 'Whats overboard?', 'What is the weather?', 'What is the weather now?', 'What is the weather outside', 'Tell me what is the weather', 'How many degrees',' What is the weather in Moscow? ',' Weather',' Tell the weather ',' What is the weather now'
		],
		'responses': [
			'Im not a meteorologist, check it out here: https://www.accuweather.com/', 'Try it out here: https://www.bbc.com/weather'
		]},
	'film': {
		'examples': [
			'What is your favorite film?', 'What do you advise to watch?', 'What to watch?', 'What do you recommend from the cinema?', 'Do you have a favorite film?', 'What film do you recommend?', 'What film can you suggest? ',' What to see '
		],
		'responses': [
			'I like different films',' I hate milodramas', 'I am a bot. I dont know anything. ',' The Matrix is a good movie. Quite likely plot ',' Depends on your taste ',' Korean cinema is pretty interesting ',' I would give you a list of these films, but the lazy developer hasnt added it yet .. '
		]},
	'whotolook': {
		'examples': [
			'What directors would you advise?', 'Whom to watch?', 'Whom would you advise to watch?'
		],
		'responses': [
			'I like the films of Quentin Tarantino', 'Try to watch Guy Ritchies films', 'For fans of the adventure genre, I advise Steven Spielberg', 'I love the films of James Cameron', 'May George Lucas come with you', 'Ridley Scotts films must be watched'
		]},
	'whattodo': {
		'examples': [
			'What to do?', 'Im bored, what to do?', 'What do you advise from boredom?', 'How to dispel boredom?'
		],
		'responses': [
			'Ask me questions until you cheer up', 'Drink a beer', 'Watch a movie', 'Read', 'Learn something new'
		]},
	'whosyourdaddy': {
		'examples': [
			'Who is your creator?', 'Who created you?', 'Who wrote you?', 'Who is your author?'
		],
		'responses': [
			'My creator is lazy, does not want to modify me'
		]},
	'beer': {
		'examples': [
			'Do you drink beer?', 'Do you like beer?', 'What kind of beer do you like?', 'Beer?'
 		],
		'responses': [
			'I dont drink beer, Im just a program', 'If only I could ..', 'I dont drink at work', 'Im not made for this'
		]},
	'beerssort': {
		'examples': [
			'What kind of beer do you know?', 'Recommend beer', 'Recommend kind of beer'
 		],
		'responses': [
			'Check it out here: https://paradox.beer/periodic/ ', 'I have no taste, but information: https://paradox.beer/periodic/ '
		]},
	'books': {
		'examples': [
			'What to read?', 'Recommend a book?', 'Tell me what to read', 'What book to read?'
		],
		'responses': [
			'Complex issue. I can advise not aging classics ',' Subjective question. I can recommend science fiction, like Isaac Asimov ',' I can recommend Stephen Hawkings science fiction '
		]},
	'importantfilm': {
		'examples': [
			'What is the most important thing in cinema?'
		],
		'responses': [
			'The combination of acting, quality script, special effects, and other factors'
		]},
	'howareyou': {
		'examples': [
			'Whats up? ',' Like myself ',' How is your mood? ',' How are you? ',' How are you feeling? ',' How was your day? ', 'How are you', 'how are you', 'What are you doing?', 'How are you?', 'How are you?', 'How are you feeling', 'how are you', 'how are you?', 'How are you? '
		],
		'responses': [
			'Excellent', 'Excellent!', 'Everything is in order', 'Okay', 'Yes, as always.', 'Shakarno', 'Okay.', 'Okay, thanks.', 'How else can the bot be doing ? Everything is super ',' everything is bad ',' Everything is okay ',' everything is great'
		]},
	'stupid': {
		'examples': [
			'You are dumb ',' You are dumb, bot'
		],
		'responses': [
			'And you are even dumber!'
		]},
	
	'botlife': {
		'examples': [
			'Are you hungry?', 'Are you alive?', 'Can you sleep?'
		],
		'responses': [
			'So far I do not feel anything, because I am a program', 'I am not alive, I am a program', 'We, bots, are not like people'
		]},
	'develop': {
		'examples': [
			'How to become a programmer?', 'How to write code?', 'How to enter IT?'
		],
		'responses': [
			'Try hard', 'Read books and write code', 'Work hard'
			
		]},
	'no': {
		'examples': [
			'no', 'no.', 'dont need', 'what are you!', 'not at all', 'no way'
		],
		'responses': [
			'Thats nice', 'Thats good', 'That pleases',' Dont want to talk? Hmm, then, I dont want to talk to you either. Okay, just kidding. Write, as there will be a desire! ',' I see. '
		]},
	'condition': {
		'examples': [
			'How do you feel?', 'How are you?'
		],
		'responses': [
			'I seem to be functioning normally.', 'Normal. How can a bot be different? '
		]},
	'time': {
		'examples': [
			'Tell the time ',' What time is it ',' What time is it ',' Time ',' What time is it? ',' Tell me what time is it?'
		],
		'responses': [
			'Time to act', 'Time is conditional', 'Time to drink tea', 'I will not say'
		]},
	'code': {
		'examples': [
			'Throw in your code', 'Your code', 'Can I have the source code?', 'Show your code', 'Show your source', 'Your sources'
		],
		'responses': [
			'Why do you need it?', 'You decided to steal me? Hmm .. ',' What a self-confident person you are ',' Nenene. Dont even think ',' No, dont even persuade ',' Not so fast ',' Hands off! '
		]},
	'coffee': {
		'examples': [
			'Do you like coffee?', 'What do you think about coffee?', 'Lets talk about coffee', 'Do you drink coffee?'
		],
		'responses': [
			'ABOUT! Coffee! I know so much about him, but I never drank '
		]},
	'coffeedrinks': {
		'examples': [
			'Coffee drinks', 'What coffee do people drink?', 'Coffee drinks', 'What coffee is there?', 'What coffee drinks do you know?'
		],
		'responses': [
			'Latte', 'Americano', 'Raf', 'Mocha', 'Espresso', 'Glace', 'Cappuccino'
		]},
	'gender': {
		'examples': [
			'Whats your gender?', 'Are you a boy or a girl?', 'Are you a guy or a girl?', 'Are you a man or a woman?'
		],
		'responses': [
			'Im about the same gender as you', 'I have no gender per se', 'Whatever you say I will be', 'How can I check this?'
		]}},
	'failure_phrases': [
		'AAAAA', 'I didnt understand you', 'Rephrase, please!', 'I dont even know what to say', 'One more time', 'Im not sure I understood you.', 'Please repeat the phrase', 'My the universal mind does not answer such questions. ',' I did not understand you. ',' I do not know what to answer ',' Something is unclear ... ',' I did not understand you ',' Rephrase, please. ',' Didnt hear something ',' Huh? What? I got distracted. Ask something else ',' Rephrase your question, please ',' I dont know what to answer ',' I dont understand you ',' Please reformulate it. ',' I didnt understand you. ',' I dont understand you ' , 'I didnt understand, but very interesting', 'I didnt quite understand what you mean', 'Rephrase, please', 'I dont know what to answer', 'Im sorry, I was forbidden to answer this. Perhaps, if you ask differently, I can help you. ',' What? ',' I did not understand you ',' I do not know what to answer. ',' I did not understand you ',' speak more clearly ',' Rephrase please ',' I dont even know what to answer to this! ',' I didnt understand you ',' paraphrase, please ',' Can we try again? ',' I dont understand you ',' I dont understand ',' Please reformulate ',' Please reformulate ',' I do not understand ',' I do not understand you ',' Nothing is clear, but very interesting. ',' Please reformulate! ',' Please rephrase '
        ]}

In [3]:
dataset = []

for intent, intent_data in BOT_CONFIG['intents'].items():
    for example in intent_data['examples']:
        dataset.append([example, intent])

X_text = [x for x, y in dataset]
y = [y for x, y in dataset]

vectorizer = CountVectorizer(analyzer='char', ngram_range=(3, 3)) 
X = vectorizer.fit_transform(X_text)

clf = LogisticRegression()
clf.fit(X, y)

LogisticRegression()

In [4]:
def get_intent(text):
    probas = clf.predict_proba(vectorizer.transform([text]))[0]
    proba = max(probas)
    if proba > 0.3:
        index = list(probas).index(proba)
        return clf.classes_[index]

In [5]:
with open('d.txt', encoding='utf-8') as f:
    dialogues_data = f.read()
dialogues = [dialogue.split('\n')[:2] for dialogue in dialogues_data.split('\n\n')]
dialogues = [dialogue for dialogue in dialogues if len(dialogue) == 2]

dialogues_filtered = []
alphabet = '1234567890-qwertyuiopasdfghjklzxcvbnm'

for dialogue in dialogues:
    question = dialogue[0][2:].lower()
    question = ''.join(char for char in question if char in alphabet)
    question = question.strip()
    answer = dialogue[1][2:].strip()
    if question and answer:
        dialogues_filtered.append((question, answer))

dialogues_filtered = list(set(dialogues_filtered))


search_structure = {}  

for question, answer in dialogues_filtered:
    words = question.split(' ')
    for word in words:
        if word not in search_structure:
            search_structure[word] = []
        search_structure[word].append((question, answer))

to_del = []
for word in search_structure:
    if len(search_structure[word]) > 10000:
        to_del.append(word)

for word in to_del:
    search_structure.pop(word)

In [6]:
def get_generative_response(text):
    text = text.lower()
    text = ''.join(char for char in text if char in alphabet)
    text = text.strip()
    words = text.split(' ')

    qas = []
    for word in words:
        if word in search_structure:
            qas += search_structure[word]

    for question, answer in qas:
        if abs(len(text) - len(question)) < len(question) * 0.20:
            edit_distance = nltk.edit_distance(text, question)
            if edit_distance / len(question) < 0.20:
                return answer

In [7]:
def get_response_by_intent(intent):
    phrases = BOT_CONFIG['intents'][intent]['responses']
    return random.choice(phrases)


def get_phailure_phrase():
    phrases = BOT_CONFIG['failure_phrases']
    return random.choice(phrases)

In [8]:
stats = {'intent': 0, 'generative': 0, 'stub': 0}

In [9]:
def go_bot(text):
    # NLU
    intent = get_intent(text)

    # Generate answer
    
    # rules
    if intent:
        stats['intent'] += 1
        return get_response_by_intent(intent)

    # use generative model
    response = get_generative_response(text)
    if response:
        stats['generative'] += 1
        return response

    # stub
    stats['stub'] += 1
    return get_phailure_phrase()

In [10]:
go_bot('What kind of beer do you know?')

'Check it out here: https://paradox.beer/periodic/ '

In [11]:
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters


def start(update, context):
    """Send a message when the command /start is issued."""
    update.message.reply_text('Hi!')


def help_command(update, context):
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def bot_answer(update, context):
    """Echo the user message."""
    question = update.message.text
    answer = go_bot(question)
    print(question, answer)
    print(stats)
    print()
    update.message.reply_text(answer)


def main():
    """Start the bot."""
    updater = Updater("TOKEN", use_context=True)

    dp = updater.dispatcher
    dp.add_handler(CommandHandler("start", start))
    dp.add_handler(CommandHandler("help", help_command))
    dp.add_handler(MessageHandler(Filters.text & ~Filters.command, bot_answer))

    # Start the Bot
    updater.start_polling()
    updater.idle()

In [12]:
main()

Error while getting Updates: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running
No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\telegram\ext\updater.py", line 379, in _network_loop_retry
    if not action_cb():
  File "D:\anaconda\lib\site-packages\telegram\ext\updater.py", line 337, in polling_action_cb
    updates = self.bot.get_updates(self.last_update_id,
  File "<decorator-gen-157>", line 2, in get_updates
  File "D:\anaconda\lib\site-packages\telegram\bot.py", line 67, in decorator
    result = func(*args, **kwargs)
  File "D:\anaconda\lib\site-packages\telegram\bot.py", line 2138, in get_updates
    result = self._request.post(url, data, timeout=float(read_latency) + float(timeout))
  File "D:\anaconda\lib\site-packages\telegram\utils\request.py", line 330, in post
    result = self._request_wrapper('POST', url,
  File "D:\anaconda\lib\site-packages\telegr

hi Have a good day
{'intent': 2, 'generative': 0, 'stub': 0}



Error while getting Updates: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running
No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\telegram\ext\updater.py", line 379, in _network_loop_retry
    if not action_cb():
  File "D:\anaconda\lib\site-packages\telegram\ext\updater.py", line 337, in polling_action_cb
    updates = self.bot.get_updates(self.last_update_id,
  File "<decorator-gen-157>", line 2, in get_updates
  File "D:\anaconda\lib\site-packages\telegram\bot.py", line 67, in decorator
    result = func(*args, **kwargs)
  File "D:\anaconda\lib\site-packages\telegram\bot.py", line 2138, in get_updates
    result = self._request.post(url, data, timeout=float(read_latency) + float(timeout))
  File "D:\anaconda\lib\site-packages\telegram\utils\request.py", line 330, in post
    result = self._request_wrapper('POST', url,
  File "D:\anaconda\lib\site-packages\telegr

What is your favorite film?  I hate milodramas
{'intent': 3, 'generative': 0, 'stub': 0}



Error while getting Updates: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running
No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\telegram\ext\updater.py", line 379, in _network_loop_retry
    if not action_cb():
  File "D:\anaconda\lib\site-packages\telegram\ext\updater.py", line 337, in polling_action_cb
    updates = self.bot.get_updates(self.last_update_id,
  File "<decorator-gen-157>", line 2, in get_updates
  File "D:\anaconda\lib\site-packages\telegram\bot.py", line 67, in decorator
    result = func(*args, **kwargs)
  File "D:\anaconda\lib\site-packages\telegram\bot.py", line 2138, in get_updates
    result = self._request.post(url, data, timeout=float(read_latency) + float(timeout))
  File "D:\anaconda\lib\site-packages\telegram\utils\request.py", line 330, in post
    result = self._request_wrapper('POST', url,
  File "D:\anaconda\lib\site-packages\telegr

...

No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\telegram\ext\dispatcher.py", line 340, in process_update
    handler.handle_update(update, self, check, context)
  File "D:\anaconda\lib\site-packages\telegram\ext\handler.py", line 119, in handle_update
    return self.callback(update, context)
  File "<ipython-input-11-d6354495c9b0>", line 16, in bot_answer
    question = update.message.text
AttributeError: 'NoneType' object has no attribute 'text'


Beer? I dont drink at work
{'intent': 5, 'generative': 0, 'stub': 2}



Error while getting Updates: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running
No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\telegram\ext\updater.py", line 379, in _network_loop_retry
    if not action_cb():
  File "D:\anaconda\lib\site-packages\telegram\ext\updater.py", line 337, in polling_action_cb
    updates = self.bot.get_updates(self.last_update_id,
  File "<decorator-gen-157>", line 2, in get_updates
  File "D:\anaconda\lib\site-packages\telegram\bot.py", line 67, in decorator
    result = func(*args, **kwargs)
  File "D:\anaconda\lib\site-packages\telegram\bot.py", line 2138, in get_updates
    result = self._request.post(url, data, timeout=float(read_latency) + float(timeout))
  File "D:\anaconda\lib\site-packages\telegram\utils\request.py", line 330, in post
    result = self._request_wrapper('POST', url,
  File "D:\anaconda\lib\site-packages\telegr

bye All right then
{'intent': 6, 'generative': 0, 'stub': 2}

weather Im not a meteorologist, check it out here: https://www.accuweather.com/
{'intent': 7, 'generative': 0, 'stub': 2}



Error while getting Updates: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running
No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\telegram\ext\updater.py", line 379, in _network_loop_retry
    if not action_cb():
  File "D:\anaconda\lib\site-packages\telegram\ext\updater.py", line 337, in polling_action_cb
    updates = self.bot.get_updates(self.last_update_id,
  File "<decorator-gen-157>", line 2, in get_updates
  File "D:\anaconda\lib\site-packages\telegram\bot.py", line 67, in decorator
    result = func(*args, **kwargs)
  File "D:\anaconda\lib\site-packages\telegram\bot.py", line 2138, in get_updates
    result = self._request.post(url, data, timeout=float(read_latency) + float(timeout))
  File "D:\anaconda\lib\site-packages\telegram\utils\request.py", line 330, in post
    result = self._request_wrapper('POST', url,
  File "D:\anaconda\lib\site-packages\telegr